# Setting up the Environment


### Mount the Drive

In [1]:
from google.colab import drive
drive.mount('gdrive')

# the project folder
%cd gdrive/'My Drive'/football_detection/

Mounted at gdrive
/content/gdrive/My Drive/football_detection


### Install Related Libraries

In [2]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -qq Cython contextlib2 pillow matplotlib pycocotools



Selecting previously unselected package python-bs4.
(Reading database ... 144676 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unpack .../5-python-html5lib_0.999999999-1_all.deb ...
Unpacking pyt

### Import Related Libraries

In [3]:
from __future__ import division, print_function, absolute_import
import pandas as pd
import numpy as np
import csv
import re
import os
import io
import glob
import shutil
#import urlib.request
import tarfile
import xml.etree.ElementTree as ET
#import tensorflow.compat.v1 as tf
%tensorflow_version 1.x
import tensorflow as tf

import cv2
from PIL import Image
from collections import namedtuple,OrderedDict
from google.colab import files
print(tf.__version__) ##1.15.0 needed

TensorFlow 1.x selected.
1.15.2


### Download tensorflow models dir

In [ ]:
#download models 
!git clone --q https://github.com/tensorflow/models.git

fatal: destination path 'models' already exists and is not an empty directory.


### Add objection detection API and other required files to path variables

In [4]:
%cd ./models/research/
#compiles the proto buffers
!protoc object_detection/protos/*.proto --python_out=.
!pip install tf_slim

# exports PYTHONPATH environment var with research and slim paths
os.environ['PYTHONPATH']+=':./:./slim/'

/content/gdrive/My Drive/football_detection/models/research
     |████████████████████████████████| 358kB 8.0MB/s 


### Testing the API

In [5]:
#testing model builder
!python3 object_detection/builders/model_builder_test.py



# Partitioning The Dataset

In [ ]:
!python partition_dataset.py -x -i Temp_Dataset/ -r 0.15

# Converting the records to tfrecords

In [ ]:
## change the pwd
os.chdir('../../')
!ls

In [ ]:
# Create train data:
!python generate_tfrecord.py -x dataset/train -l tf_records/label_map.pbtxt -o tf_records/train.record

# Create test data:
!python generate_tfrecord.py -x dataset/test -l tf_records/label_map.pbtxt -o tf_records/test.record


Successfully created the TFRecord file: tf_records/train.record
Successfully created the TFRecord file: tf_records/test.record


# Training 

In [ ]:
## run only if converting to tf records have been run
## checking the pwd
!ls
## changing pwd to research again
os.chdir('models/research')

dataset		      object_detection	      training_quant
generate_tfrecord.py  pretrained_model_quant  training_script.ipynb
models		      tf_records


### Downloading and using ngrok for tensorboard hosting

In [9]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2020-09-27 07:30:10--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.206.15.164, 3.222.114.249, 52.72.152.212, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.206.15.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.13M  12.7MB/s    in 1.0s    

2020-09-27 07:30:12 (12.7 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [14]:
#the logs that are created while training 
LOG_DIR = "../../training_quant/"
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6007 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6007 &')
#The link to tensorboard.
#works after the training starts.
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://9b7ef0edd4fe.ngrok.io


### Running the training script

In [15]:
#!python3 ./object_detection/model_main.py --pipeline_config_path='./object_detection/samples/configs/ssd_mobilenet_v1_coco_istoc.config' --model_dir=training/
!python3 ./object_detection/model_main.py --pipeline_config_path='./object_detection/samples/configs/ssd_mobilenet_v2_quantized_300x300_coco_football.config' --model_dir=../../training_quant/


W0927 07:59:38.507576 140087724603264 model_lib.py:771] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: None
I0927 07:59:38.507875 140087724603264 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0927 07:59:38.507973 140087724603264 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0927 07:59:38.508058 140087724603264 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0927 07:59:38.508144 140087724603264 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0927 07:59:38.508272 140087724603264 model_lib.py:787] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_to_t

# Converting the model using TFlite Converter

In [16]:
# by Chengwei
#dir where the model will be saved
output_directory = '../../fine_tuned_quantmodel'

lst = os.listdir('../../training_quant')
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join('../../training_quant', last_model)

In [17]:
!python ./object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --input_shape=1,300,300,3 \
    --pipeline_config_path=./object_detection/samples/configs/ssd_mobilenet_v2_quantized_300x300_coco_football.config \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

Instructions for updating:
Please use `layer.__call__` method instead.
W0927 10:04:48.160334 140173023094656 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0927 10:04:50.045083 140173023094656 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0927 10:04:50.071788 140173023094656 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0927 10:04:50.097966 140173023094656 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

In [18]:
## source https://medium.com/@teyou21/convert-a-tensorflow-frozen-graph-to-a-tflite-file-part-3-1ccdb3874c4a
!python ./object_detection/export_tflite_ssd_graph.py --pipeline_config_path='object_detection/samples/configs/ssd_mobilenet_v2_quantized_300x300_coco_football.config' --trained_checkpoint_prefix={last_model_path} --output_directory=../../tflite --add_postprocessing_op=true

Instructions for updating:
Please use `layer.__call__` method instead.
W0927 10:05:21.922955 139836712327040 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0927 10:05:23.788005 139836712327040 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0927 10:05:23.816900 139836712327040 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0927 10:05:23.845182 139836712327040 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

In [19]:
##source: https://medium.com/@teyou21/convert-a-tensorflow-frozen-graph-to-a-tflite-file-part-3-1ccdb3874c4a
!tflite_convert --graph_def_file=../../tflite/tflite_graph.pb --output_file=../../tflite/detect.tflite --output_format=TFLITE --input_shapes=1,300,300,3 --input_arrays=normalized_input_image_tensor --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'  --inference_type=QUANTIZED_UINT8 --mean_values=128 --std_dev_values=127 --change_concat_input_ranges=false --allow_custom_ops

2020-09-27 10:06:08.675350: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-27 10:06:08.688156: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-27 10:06:08.688752: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:00:04.0
2020-09-27 10:06:08.689009: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-09-27 10:06:08.690884: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-09-27 10:06:08.692541: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuf